In [1]:
import requests
import re
from bs4 import BeautifulSoup
import string
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from pyshadow.main import Shadow
import pandas as pd
import csv, os
from selenium.common.exceptions import NoSuchElementException
import time

In [2]:
def Download_Citation(Volume, Issue):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #driver.execute_script("window.open('https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue)+"/');")
    driver.get("https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue))
    driver.find_element(By.CSS_SELECTOR, "a[class='bulk-download-option'] span").click()
    Research_Articles = driver.find_elements(By.CLASS_NAME,'checkbox--primary')
    for a in Research_Articles[1:]:
        a.click()
    driver.find_element(By.CSS_SELECTOR, ".bulk-wizard.exportCitationsForm").click()
    driver.find_element(By.CSS_SELECTOR, ".export-citations__submit").click()
    time.sleep(5)
    driver.quit()

In [3]:
def Ref(html_name,year): 
    Ref = []
    with open('/Users/langchen777/Desktop/NewDirectionsforEvaluations/New Directions for Evaluations/HTML/'+str(year) + "/" +html_name+'.html', 'r', encoding = 'utf-8') as f:
        soup = BeautifulSoup(f.read())
        content = soup.findAll("li")
    for li in content:
        authors = li.findAll("span", class_="author")
        line = []
        title =""
        for x in authors:
            line.append(x.contents[0])
        while(len(line)<5):
            line.append("")
        while(len(line)>5):
            line.pop()
        try:
            title = li.find("span", class_="articleTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="chapterTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="bookTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="otherTitle").text
        except AttributeError:
            pass
        line.append(title)
        Ref.append(line)
    return Ref

In [4]:
col=['AU1_r','AU2_r','AU3_r','AU4_r','AU5_r','TI_r','TY']
for i in range(1,11):
    col.append('AU'+str(i))
col = col + ["TI","JO","VL","IS","SN","DO","SP","EP"]
        #a_series = pd.Series(line, index = df.columns)
        #df = df.append(a_series, ignore_index=True)

In [5]:
def Issue(volume, issue):
    df = pd.DataFrame(columns=col)
    file = open('/Users/langchen777/Downloads/pericles_exported_citations.txt', 'r')
    Lines = file.readlines()
    count = 0
    for line in Lines:
        start = line[:2]
        if start == "TY":
            count = count + 1
            #Ref = Ref()
            row = []
            row.append(line.split("-",1)[1].strip())
        elif start == "AU":
            row.append(line.split("-",1)[1].strip())
        elif start == "TI":
            while(len(row)<11):
                row.append("")
            while(len(row)>11):
                row.pop()
            row.append(line.split("-",1)[1].strip())
        elif start in ["JO","VL","IS","SN","DO","SP","EP"]:
            row.append(line.split("-",1)[1].strip())
        elif start == "ER":
            try:
                for r in Ref(str(issue)+str(count),volume):
                    combine = r + row
                    a_series = pd.Series(combine, index = col)
                    df = df.append(a_series, ignore_index=True)
            except FileNotFoundError:
                pass
            else:
                r = ["","","","","",""]
                combine = r + row
    os.remove("/Users/langchen777/Downloads/pericles_exported_citations.txt")
    return df

In [6]:
def Return_Issue(i):
    if i in range(1,5):
        volume = 1978
    else:
        volume = 1979 + (i-1)//4
    Download_Citation(volume, i)
    df=Issue(volume,i)
    return df

In [16]:
df = pd.DataFrame(columns=col)
for i in range (1,4+1):
    df = df.append(Return_Issue(i))
df.to_csv('1978.csv')
print("Finsihed!")



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
<ipython-input-2-04f0a9228132>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/langchen777/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver v

Finsihed!


In [19]:
import glob
import pandas as pd

interesting_files = glob.glob("/Users/langchen777/Desktop/NewDirectionsforEvaluations/CSV/*.csv") 
df = pd.concat((pd.read_csv(f, header = 0) for f in interesting_files))
df.to_csv("output.csv")